### First, implement the `process_bigraph.Process` interface.
This interface requires the following methods:

- `inputs() -> Dict(InputPortSchema)`: define the input port type schemas using the Bigraph Schema type system. You may also create and register custom types, which is not shown for the sake of simplicity.
- `outputs() -> Dict(OutputPortSchema)`: define the output port type schemas using the Bigraph Schema type system. You may also create and register custom types, which is not shown for the sake of simplicity.
- `update(state: Dict(InputDict), interval: float) -> Dict(OutputDict)`: define the callback to be used to generate the data specified by the `outputs()` method.
- `initial_state() -> OutputDict`: **for processes only**: define the data to be used at iteration 0, which will be used in the update method.

We can think of the dictionaries returned by these methods as Input and Output port schemas, whose structures are specified in `inputs()` and `outputs()`. The `update()` method returns the actual data specified by the aforementioned methods.

In [12]:
class StateData(dict):
    def __new__(cls, *args, **kwargs):
        return super(StateData, cls).__new__(cls, *args, **kwargs)

class PortSchema(dict):
    def __new__(cls, *args, **kwargs):
        return super(PortSchema, cls).__new__(cls, *args, **kwargs)


class InputPortSchema(PortSchema):
    pass


class OutputPortSchema(PortSchema):
    pass


In [13]:
from typing import Any
import numpy as np
from process_bigraph import Process, Composite, ProcessTypes, pp


class Toy(Process):
    config_schema = {
        'k': 'float'
    }
    def __init__(self, config: dict[str, Any] = None, core: ProcessTypes = None) -> None:
        super().__init__(config, core)
        self.k = self.config['k']

    def initial_state(self) -> StateData:
        return StateData(
            x=self.k,
            y=self.k**self.k
        )

    def inputs(self) -> InputPortSchema:
        return InputPortSchema(
            x='float'
        )

    def outputs(self) -> OutputPortSchema:
        return OutputPortSchema(
            x='float',
            y='float'
        )

    def update(self, state, interval) -> StateData:
        x = state['x'] * self.k
        return StateData({
            'x': x,
            'y': x**x
        })

### Register the newly created process with the app-specific `ProcessTypes()` singleton; in this case which is stored in `app_registrar`. **This is the only way that the `Composite` instance (which runs the simulation) will know how to appropriately reference the pointers required.

In [14]:
from bsp import app_registrar

app_registrar.core.register_process('toy', Toy)

Registered module DatabaseConfig
Registered module GeometryConfig
Registered module MeshFileConfig
Registered module ParametersConfig
Registered module OsmoticModelConfig
Registered module SBMLFileCobraConfig
Registered module SBMLModelChangesConfig
Registered module SBMLModelConfig
Registered module TimeCourseConfig
Registered module TensionModelConfig
Registered module SedModelConfig
Registered module BoundsType
Registered module PositiveFloatType
Registered module MechanicalForcesType
Registered module ProteinDensityType
Registered module GeometryType
Registered module VelocitiesType
Registered module OsmoticParametersType
Registered module SurfaceTensionParametersType
Registered module ParticleType
Successfully registered <class 'bsp.processes.cobra_process.CobraProcess'> to cobra-process
Successfully registered <class 'bsp.processes.cobra_process.SedCobraProcess'> to sed-cobra-process
Successfully registered <class 'bsp.processes.cobra_process.DynamicFBA'> to dynamic-fba
Successfu

In [15]:
config = {
    'k': 0.35
}

state = {
    'toy': {
        '_type': 'process',
        'address': 'local:toy',
        'config': config,
        'inputs': {
            'x': ['x_store'],
        },
        'outputs': {
            'x': ['x_store'],
            'y': ['y_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'x': 'float',
                'y': 'float',
            }
        },
        'inputs': {
            'x': ['x_store'],
            'y': ['y_store'],
        }
    }
}

### Instantiate a `Composite` using the spec you just defined, and then run it. In this case the duration you specified will be parsed iteratively as a monotonically increasing range.

In [16]:
sim = Composite(config={'state': state}, core=app_registrar.core)

In [17]:
sim.run(10)

### Gather the results

In [18]:
results = sim.gather_results()

In [19]:
pp(results)

{ ('emitter',): [ {'x': 0.35, 'y': 0.6925064384421088},
                  {'x': 0.4725, 'y': 1.4657150296277588},
                  {'x': 0.637875, 'y': 2.208314187919128},
                  {'x': 0.8611312499999999, 'y': 2.9238252284962183},
                  {'x': 1.1625271874999998, 'y': 3.62047440265183},
                  {'x': 1.5694117031249997, 'y': 4.31405967575313},
                  {'x': 2.1187057992187492, 'y': 5.033635203474405},
                  {'x': 2.8602528289453115, 'y': 5.834762885376529},
                  {'x': 3.8613413190761703, 'y': 6.835852560963448},
                  {'x': 5.21281078075283, 'y': 8.338235469258873},
                  {'x': 7.03729455401632, 'y': 11.333572543663768}]}
